<a href="https://colab.research.google.com/github/shahdhesham/Colab-Thesis/blob/main/Magicoder%2BDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [ ]:
# Iterative following prompt template - NOT BEST RESULTS

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Hardcoded C Code (modify this as needed)
c_code = """
#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}
"""

# 2. Load Model (do this once)
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Core Translation Function with Structured Prompts
def translate_c_to_cpp(refinement_prompt=""):
    prompt = f"""### Instruction:
Translate the following C code to C++. Follow these rules:
1. Output ONLY raw C++ code (no Natural Language Text)
2. Maintain exact functionality
3. DO NOT include markdown (no ```cpp or ```)
4. MUST include all required headers
5. {refinement_prompt}

EXAMPLE OUTPUT:
#include <iostream>
using namespace std;

int main() {{
    cout << "Hello World";
    return 0;
}}


### C Code:
{c_code}

### Response (C++ Code):
"""  # Explicit response marker

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0,
        pad_token_id=tokenizer.eos_token_id  # Use EOS token as pad token
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract everything after the response marker
    translated_code = full_response.split("### Response (C++ Code):")[-1].strip()
    return translated_code

# 4. Interactive Refinement Loop
print("Initial C++ translation:")
initial_cpp = translate_c_to_cpp()
print(initial_cpp)

while True:
    refinement = input("\nRefinement prompt (e.g. 'use vectors', 'quit' to exit): ").strip()
    if refinement.lower() == 'quit':
        break

    refined_cpp = translate_c_to_cpp(refinement)
    print("\nRefined C++ code:")
    print(refined_cpp)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Initial C++ translation:
```cpp
#include <iostream>

int main() {
    int a, b;
    std::cout << "Enter two numbers: ";
    std::cin >> a >> b;
    std::cout << "Sum: " << a + b << std::endl;
    return 0;
}
```

Refinement prompt (e.g. 'use vectors', 'quit' to exit): remove ```cpp

Refined C++ code:
```cpp
#include <iostream>

int main() {
    int a, b;
    std::cout << "Enter two numbers: ";
    std::cin >> a >> b;
    std::cout << "Sum: " << a + b << std::endl;
    return 0;
}
```

Refinement prompt (e.g. 'use vectors', 'quit' to exit): quit


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [11]:
#read from file
from google.colab import files

uploaded = files.upload()  # Opens a file upload dialog

filename = list(uploaded.keys())[0]  # Get the uploaded file name
with open(filename, "r") as file:
    c_code = file.read()  # Read the file content


Saving s834386135.c to s834386135 (1).c


In [12]:
def save_to_file(code, base_filename, suffix):
    # Replace '.c' with '.cpp' and append the suffix for refined versions
    output_filename = base_filename.replace(".c", f"_{suffix}.cpp")
    with open(output_filename, "w") as f:
        f.write(code)
    print(f"Saved as {output_filename}")


In [13]:

# 2. Load Model (do this once)
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                           device_map="auto",
                                           torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# 1. Hardcoded C Code (modify this as needed)
# will replace it with file read

# c_code = """
# #include <stdio.h>

# int main() {
#     int a, b;
#     printf("Enter two numbers: ");
#     scanf("%d %d", &a, &b);
#     printf("Sum: %d\n", a + b);
#     return 0;
# }
# """


In [14]:
# Deepseek Iterative model - BEST RESULTS NOT FOLLOWING PROMPT TEMPLATE
#lightweight instruction-following format


# 3. Core Translation Function
def translate_c_to_cpp(refinement_prompt=""):
    prompt = f"""
    Translate this C to C++.
     Rules:
    1. Output ONLY raw C++ code
    2. {refinement_prompt}

    C:
    {c_code}

    C++:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("C++:")[-1].strip()



In [15]:
# 4. Interactive Refinement Loop
#print("Initial C translation:")
initial_cpp =  translate_c_to_cpp()
print(initial_cpp)

save_to_file(initial_cpp, filename)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#include<iostream>
#include<cmath>

int main()
{
    int a,b,sum,i;
    while(std::cin >> a >> b)
    {
        sum=a+b;
        i=1;
        do
        {
            sum=sum/10;
            i++;
        }while(sum!=0);
        std::cout << i-1 << std::endl;
    }
    return 0;
}
Saved as s834386135 (1)_initial.cpp


In [16]:
#refinment loop
while True:

    refinement = input("\nRefinement prompt  or 'quit': ")
    if refinement.lower() == 'quit':
        break

    refined_cpp = translate_c_to_cpp(refinement)
    #print("\nRefined C code:")
    print(refined_cpp)
        # Save the refined translation
    save_to_file(refined_cpp, filename)


Refinement prompt  or 'quit': change variables to s and Z


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#include<iostream>
#include<cmath>

int main()
{
    int s, Z;
    while(std::cin >> s >> Z)
    {
        int i=1;
        int sum=s+Z;
        do
        {
            sum=sum/10;
            i++;
        }while(sum!=0);
        std::cout << i-1 << std::endl;
    }
    return 0;
}
Saved as s834386135 (1)_refined.cpp

Refinement prompt  or 'quit': quit
